In [1]:
import pandas as pd
import scipy as sc
from scipy.sparse import *
import numpy as np
from sklearn.neighbors import NearestNeighbors
import time
import pickle
import csv
import sklearn

In [ ]:
#dont run anytime

df1 = pd.read_csv('events_train.csv', low_memory=False)
df2 = pd.read_csv('purchases_train.csv', low_memory=False)
frames = [df1, df2]
merged = pd.concat(frames)
merged = merged.sort_values('timestamp')
merged.to_csv('sorted.csv', index=False)

In [2]:
df1 = pd.read_csv('sorted.csv', low_memory=False)
df1.loc[df1['event_type'] == 'view_item', 'event_type'] = 1
df1.loc[df1['event_type'] == 'purchase_item', 'event_type'] = 3

In [3]:
customers_dict={}
reverse_customers_dict={}
customer_counter=0

for index, row in enumerate(df1.customer_id.unique()):
    customers_dict[row] = index
    reverse_customers_dict[index]= row

users=df1.customer_id.values
for index,row in enumerate(users):
        users[index]=customers_dict.get(users[index])  

In [4]:
products_dict={}
reverse_products_dict={}
product_counter=0

for index, row in enumerate(df1.product_id.unique()):
    products_dict[row] = index
    reverse_products_dict[index]=row

products=df1.product_id.values
for index,row in enumerate(products):
        products[index]=products_dict.get(products[index])

In [5]:
matrix = csr_matrix((df1.event_type.values, (df1.customer_id.values, df1.product_id.values)))

In [6]:
with open('vi_challenge_uID.csv') as f:
    content = f.readlines()
content = [x.strip() for x in content] 

In [7]:
usersdf2= content
users_to_generate=[]
new_users=[]
for index,row in enumerate(usersdf2):
    new_index=customers_dict.get(usersdf2[index])
    if new_index == None:
        new_users.append(row)
    else:
        users_to_generate.append(new_index)
test_customers = users_to_generate
print(len(test_customers))
print(len(new_users))

850
150


In [8]:
pocet_podobnych_pouzivatelov=5

In [9]:
neigh = NearestNeighbors(n_neighbors=pocet_podobnych_pouzivatelov)
neigh.fit(matrix)
start = time.time()
knn=neigh.kneighbors(matrix[test_customers])
end = time.time()
print(end - start)
print(knn)

71.01942253112793
(array([[ 0.        ,  5.83095189,  6.        ,  6.244998  ,  6.55743852],
       [ 0.        ,  7.87400787,  8.        ,  8.        ,  8.        ],
       [ 0.        ,  1.73205081,  2.        ,  2.        ,  2.        ],
       ...,
       [ 0.        ,  1.73205081,  1.73205081,  1.73205081,  1.73205081],
       [ 0.        ,  0.        ,  1.        ,  1.        ,  1.        ],
       [ 0.        , 15.77973384, 17.52141547, 17.80449381, 18.05547009]]), array([[1986410, 1834471, 2186012, 1771292, 1863765],
       [ 811899, 1218948,  806525, 1058304, 1316331],
       [ 634162,  761372, 1132893, 2068092,  373243],
       ...,
       [1909128,  423955,  469775, 1987241,  552171],
       [1012394,  998461,  924398,  994762, 1032861],
       [1711461, 1623881, 1501302, 1122323,   47176]], dtype=int64))


In [10]:
top_products_dict={}

for index, row in enumerate(df1.product_id.unique()):
    top_products_dict[row] = 0

products=df1.product_id.values
for index,row in enumerate(products):
        top_products_dict[row]+=1
top_products_dict=sorted(top_products_dict, key=top_products_dict.__getitem__, reverse = True)
top_products_dict=top_products_dict[:10]

top_productsID=[]
for product in top_products_dict:
    top_productsID.append(reverse_products_dict[product])

In [11]:
dataTocsv = []
a='customer_id'
b='product_id'
dataTocsv.append([a,b])
print(dataTocsv)

[['customer_id', 'product_id']]


In [12]:
number_of_product_to_recomended=10

for index, test_customer in enumerate(test_customers):
    to_recomend=[]
    torecomend_dict={}
    zipped=zip(knn[0][index],knn[1][index])
    last_similarity=-1
    weight_of_user=pocet_podobnych_pouzivatelov*2
    
    for similarity,indicies in zipped:
        if last_similarity==-1:
            last_similarity=similarity
        if similarity!=last_similarity:
            weight_of_user -= 2
            last_similarity = similarity
        nieco = matrix[indicies].toarray().nonzero()[1]
        
        for product in nieco:
                to_recomend.append(product)
                if product in torecomend_dict:                
                    torecomend_dict[product]+=matrix[indicies,product]*weight_of_user
                else:
                    torecomend_dict[product]=matrix[indicies,product]*weight_of_user

    recomended=sorted(torecomend_dict, key=torecomend_dict.__getitem__, reverse = True)
    recomendsID=[]
    
    for index,item in enumerate(recomended):
        if index>= 10:        
            break  
        recomendsID.append(reverse_products_dict[item])
        dataTocsv.append([reverse_customers_dict[test_customer],reverse_products_dict[item]])

    if len(recomendsID)<=10:
        pocet=10-len(recomendsID)
        for i in range(pocet):
            dataTocsv.append([reverse_customers_dict[test_customer],top_productsID[i]])

In [13]:
for user in new_users:
    for index in range(10):
        dataTocsv.append([user,top_productsID[index]])

In [14]:
with open("submission36.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(dataTocsv)